## Deliverable 2. Create a Customer Travel Destinations Map.

In [111]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [151]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()
#city_data_df.count()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Conditions,Date
0,0,Tiarei,PF,-17.53,-149.33,73.40,83,20,2.24,few clouds,2020-10-31 16:23:03
1,1,East London,ZA,-33.02,27.91,66.20,63,20,9.17,few clouds,2020-10-31 16:23:04
2,2,San Cristobal,VE,7.77,-72.22,82.40,74,75,4.70,broken clouds,2020-10-31 16:23:04
3,3,Ushuaia,AR,-54.80,-68.30,55.40,54,40,3.36,scattered clouds,2020-10-31 16:21:01
4,4,Yecuatla,MX,19.87,-96.75,74.57,51,0,3.67,clear sky,2020-10-31 16:23:04


In [165]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [166]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Conditions,Date
0,0,Tiarei,PF,-17.53,-149.33,73.40,83,20,2.24,few clouds,2020-10-31 16:23:03
2,2,San Cristobal,VE,7.77,-72.22,82.40,74,75,4.70,broken clouds,2020-10-31 16:23:04
4,4,Yecuatla,MX,19.87,-96.75,74.57,51,0,3.67,clear sky,2020-10-31 16:23:04
13,13,Praia Da Vitoria,PT,38.73,-27.07,73.40,83,75,17.96,broken clouds,2020-10-31 16:23:08
16,16,Vila Velha,BR,-20.33,-40.29,73.40,88,75,2.24,light rain,2020-10-31 16:23:09
...,...,...,...,...,...,...,...,...,...,...,...
671,671,Kloulklubed,PW,7.04,134.26,84.20,74,90,10.29,moderate rain,2020-10-31 16:26:16
676,676,Sokoni,TZ,-6.43,39.55,82.40,69,40,6.93,scattered clouds,2020-10-31 16:26:17
677,677,Khanpur,PK,28.65,70.66,71.92,20,0,4.65,clear sky,2020-10-31 16:26:18
678,678,Vanimo,PG,-2.67,141.30,78.21,85,7,5.82,clear sky,2020-10-31 16:26:18


In [167]:
# 4a. Determine if there are any empty rows.
# using count instead of isnull easier to see if missing values on large dataset.
preferred_cities_df.count()

City_ID               271
City                  271
Country               266
Lat                   271
Lng                   271
Max Temp              271
Humidity              271
Cloudiness            271
Wind Speed            271
Current Conditions    271
Date                  271
dtype: int64

In [168]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID               266
City                  266
Country               266
Lat                   266
Lng                   266
Max Temp              266
Humidity              266
Cloudiness            266
Wind Speed            266
Current Conditions    266
Date                  266
dtype: int64

In [169]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Conditions", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)
#hotel_df.count()

,City,Country,Max Temp,Current Conditions,Lat,Lng,Hotel Name
0,Tiarei,PF,73.40,few clouds,-17.53,-149.33,
2,San Cristobal,VE,82.40,broken clouds,7.77,-72.22,
4,Yecuatla,MX,74.57,clear sky,19.87,-96.75,
13,Praia Da Vitoria,PT,73.40,broken clouds,38.73,-27.07,
16,Vila Velha,BR,73.40,light rain,-20.33,-40.29,
21,Vaini,TO,71.60,broken clouds,-21.20,-175.20,
22,Faanui,PF,78.57,scattered clouds,-16.48,-151.75,
25,San Patricio,US,71.60,smoke,28.02,-97.52,
34,Robertsganj,IN,71.01,scattered clouds,24.70,83.07,
36,Hithadhoo,MV,83.84,overcast clouds,-0.60,73.08,


In [170]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")
    
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [175]:
# 7 drop any rows in the hotel_df DataFrame where a hotel name is not found.
#clean_hotel_df = hotel_df.dropna(how = "any")
hotel_df.count()

City                  266
Country               266
Max Temp              266
Current Conditions    266
Lat                   266
Lng                   266
Hotel Name            266
dtype: int64

In [171]:
# dropna() did not identify the locations skipped; empty hotel names
# pull the index for the empty hotel names and manually drop

index_names = hotel_df[hotel_df['Hotel Name'] == '' ].index 
#print(index_names)
  
# drop these row indexes from dataFrame 
clean_hotel_df = hotel_df.drop(index_names) 
clean_hotel_df

,City,Country,Max Temp,Current Conditions,Lat,Lng,Hotel Name
0,Tiarei,PF,73.40,few clouds,-17.53,-149.33,Le Rocher de Tahiti
2,San Cristobal,VE,82.40,broken clouds,7.77,-72.22,Pirineos
4,Yecuatla,MX,74.57,clear sky,19.87,-96.75,La Esperanza Yecuatla Veracruz
13,Praia Da Vitoria,PT,73.40,broken clouds,38.73,-27.07,Hotel Branco I
16,Vila Velha,BR,73.40,light rain,-20.33,-40.29,Hotel Vitória Palace
...,...,...,...,...,...,...,...
669,Canutama,BR,84.09,moderate rain,-6.53,-64.38,Hotel Jhennifer
671,Kloulklubed,PW,84.20,moderate rain,7.04,134.26,Storyboard Beach Resort
676,Sokoni,TZ,82.40,scattered clouds,-6.43,39.55,Ujamaa Beach Resort
677,Khanpur,PK,71.92,clear sky,28.65,70.66,VIP shama biryani khanpur


In [176]:
clean_hotel_df.count()

City                  242
Country               242
Max Temp              242
Current Conditions    242
Lat                   242
Lng                   242
Hotel Name            242
dtype: int64

In [172]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [173]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Conditions</dt><dd>{Current Conditions}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [174]:
# 11a. Add a marker layer for each city to the map. 
#marker_layer = gmaps.marker_layer(locations)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))